In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold
from statsmodels.graphics.mosaicplot import mosaic
import pyarrow
import os
from datetime import datetime
currWD = os.getcwd()
print("Current Working Directory:", currWD)
# os.chdir("Set WD here")
filepath = "./data/catB_train.parquet"
data = pd.read_parquet(filepath, engine='pyarrow')

Current Working Directory: C:\Users\rocki\OneDrive\Desktop\NUS Datathon2 2024\Datathon\Swislar


In [237]:
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter


In [65]:
#Keep an original copy of the data 

#df = data.copy()
data = df.copy()

In [66]:
data['cltdob_fix'] = data['cltdob_fix'].replace("None", pd.NaT)
data['cltdob_fix'] = pd.to_datetime(data['cltdob_fix'])

data['min_occ_date'] = data['min_occ_date'].replace("None", pd.NaT)
data['min_occ_date'] = pd.to_datetime(data['min_occ_date'])


cltdob_rows = data[data['cltdob_fix'].isna()]
data = data.dropna(subset=['cltdob_fix'])
# Drop rows with incomplete data for `min_occ_date`
occDate_rows = data[data['min_occ_date'].isna()]
data = data.dropna(subset=['min_occ_date'])

# Convert cltdob_fix to datetime format and compute the client's age
currentDate = datetime.now()
data["cltdob_fix"] = pd.to_datetime(data["cltdob_fix"])
data["cltage"] = data["cltdob_fix"].map(
    lambda x: ((currentDate - x).days/365.25))
# Convert min_occ_date to datetime format and compute cltage_start
data["min_occ_date"] = pd.to_datetime(data["min_occ_date"])
data["cltage_start"] = (
    (data["min_occ_date"] - data["cltdob_fix"]).dt.days/365.25).round().astype(int)

# Computing the customer tenure
data["clt_ten"] = data["cltage"] - data["cltage_start"]
# Adding back NA rows
data = pd.concat([data, cltdob_rows, occDate_rows], axis=0)
# Coverting NA to 0 for "f_purchase_lh" col
data["f_purchase_lh"] = np.where(data["f_purchase_lh"].isna(), 0, 1)
# Coverting NA to 0 for "is_dependent_in_at_least_1_policy" col
data["is_dependent_in_at_least_1_policy"] = np.where(
    data["is_dependent_in_at_least_1_policy"].isna(), 0, 1)

In [68]:
data01 = data.copy()
x = data.dtypes[data.dtypes == object].index.to_list()

In [70]:
for i in x:
    try:
        data01[i] = pd.to_numeric(data01[i])
    except:
        data01[i] = data01[i]

In [77]:
data01.drop(columns = ["clntnum"], inplace = True)

In [209]:
data02 = data01.copy()

In [210]:
y = data01.dtypes[data01.dtypes == "object"].index.to_list()

In [211]:
data01[y].nunique()

race_desc             4
ctrycode_desc        26
clttype               3
stat_flag             3
cltsex_fix            2
hh_size_est           6
annual_income_est     5
dtype: int64

In [212]:
enc = LabelEncoder()
data02[y] = data01[y].apply(lambda x : enc.fit_transform(x))

In [213]:
all_missing = data02.isna().sum()[data02.isna().sum() > 17992/2].index.to_list()
data02.drop(columns = all_missing, inplace = True)

In [221]:
imputer = KNNImputer(n_neighbors= 5)
data02["min_occ_date"] = data02["min_occ_date"].values.astype("float64")
data02["cltdob_fix"] = data02["cltdob_fix"].values.astype("float64")
# datetime = data02[["min_occ_date", "cltdob_fix"]]
# data02.drop(columns = ["min_occ_date", "cltdob_fix"], inplace = True)
imputer.fit_transform(data02)

array([[ 0.        , 16.        ,  2.        , ..., 49.72210815,
        43.        ,  6.72210815],
       [ 0.        , 16.        ,  2.        , ..., 44.21355236,
        28.        , 16.21355236],
       [ 3.        , 16.        ,  2.        , ..., 48.        ,
        44.        ,  4.        ],
       ...,
       [ 4.        , 16.        ,  1.        , ..., 88.85804244,
        29.        , 59.85804244],
       [ 4.        , 16.        ,  1.        , ..., 90.72306639,
        30.4       , 60.32306639],
       [ 4.        , 16.        ,  1.        , ..., 90.72306639,
        30.4       , 60.32306639]])

In [222]:
transformed = imputer.fit_transform(data02)

In [223]:
data03 = pd.DataFrame(transformed, columns = imputer.get_feature_names_out())

In [224]:
data03

,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,...,n_months_last_bought_lh_f852af,n_months_last_bought_lh_947b15,n_months_last_bought_32c74c,f_elx,f_mindef_mha,f_retail,f_purchase_lh,cltage,cltage_start,clt_ten
0,0.0,16.0,2.0,0.0,1.509408e+18,1.372896e+17,0.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,0.0,0.0,1.0,0.0,49.722108,43.0,6.722108
1,0.0,16.0,2.0,0.0,1.179878e+18,3.111264e+17,1.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,0.0,0.0,1.0,0.0,44.213552,28.0,16.213552
2,3.0,16.0,2.0,0.0,1.567210e+18,1.916352e+17,1.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,0.0,0.0,1.0,0.0,48.000000,44.0,4.000000
3,0.0,16.0,2.0,0.0,1.634515e+18,1.960416e+17,0.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,0.0,0.0,1.0,0.0,47.860370,46.0,1.860370
4,0.0,16.0,2.0,0.0,1.532045e+18,8.071488e+17,0.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,0.0,0.0,1.0,0.0,28.495551,23.0,5.495551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17987,4.0,16.0,1.0,1.0,-9.223372e+18,7.155648e+17,1.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,1.0,0.0,0.0,0.0,90.723066,30.4,60.323066
17988,4.0,16.0,1.0,1.0,-9.223372e+18,7.819200e+17,1.0,0.0,0.0,0.0,...,9999.0,9999.0,26.0,0.0,0.0,1.0,0.0,90.723066,30.4,60.323066
17989,4.0,16.0,1.0,1.0,-9.223372e+18,8.337600e+17,0.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,1.0,0.0,0.0,0.0,88.858042,29.0,59.858042
17990,4.0,16.0,1.0,1.0,-9.223372e+18,7.791552e+17,1.0,0.0,0.0,0.0,...,9999.0,9999.0,9999.0,1.0,0.0,0.0,0.0,90.723066,30.4,60.323066


In [226]:
data03 = pd.concat([data03,pd.Series(data02.index)], axis = 1)

In [227]:
data03.set_index(0, inplace = True)

In [238]:
y = data03["f_purchase_lh"]
X = data03.drop(columns=['f_purchase_lh'])

In [240]:
oversample = SMOTE(sampling_strategy = 0.1)
undersample = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', oversample), ('u', undersample)]
pipeline = Pipeline(steps=steps)

In [241]:
counter = Counter(y)
print(counter)
X, y = pipeline.fit_resample(X, y)
counter = Counter(y)
print(counter)

Counter({0.0: 17282, 1.0: 710})
Counter({0.0: 3456, 1.0: 1728})


In [249]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [262]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn import metrics

def RFE_and_model_evaluation(model):
    min_features_to_select = 1  # Minimum number of features to consider
    clf = model
    cv = StratifiedKFold(5)
    #    cv = RepeatedStratifiedKFold(random_state = 5)


    rfecv = RFECV(
        estimator=clf,
        step=1,
        cv=cv,
        scoring="f1_macro",
        min_features_to_select=min_features_to_select,
        n_jobs=2,
    )
    rfecv.fit(X_train, y_train)

    print(f"Optimal number of features: {rfecv.n_features_}")
    print(rfecv.get_feature_names_out())
    print(rfecv.score(X_val,y_val))

In [263]:
RFE_and_model_evaluation(LogisticRegression())

Optimal number of features: 2
['min_occ_date' 'cltdob_fix']
0.6576663452266153


In [264]:
RFE_and_model_evaluation(DecisionTreeClassifier())

Optimal number of features: 49
['stat_flag' 'min_occ_date' 'cltdob_fix' 'cltsex_fix' 'is_consent_to_mail'
 'is_consent_to_email' 'is_consent_to_sms' 'is_valid_dm' 'is_valid_email'
 'is_housewife_retiree' 'is_class_1_2' 'is_dependent_in_at_least_1_policy'
 'hh_20' 'pop_20' 'hh_size' 'hh_size_est' 'annual_income_est'
 'n_months_last_bought_products' 'flg_latest_being_lapse'
 'tot_inforce_pols' 'ape_ltc_43b9d5' 'ape_lh_839f8a' 'ape_lh_e22a6a'
 'ape_grp_70e1dd' 'ape_lh_947b15' 'sumins_grp_1581d7' 'sumins_grp_70e1dd'
 'sumins_lh_f852af' 'prempaid_lh_f852af' 'ape_507c37' 'f_hold_839f8a'
 'sumins_ltc' 'sumins_507c37' 'n_months_last_bought_e22a6a'
 'n_months_last_bought_507c37' 'n_months_last_bought_gi'
 'f_ever_bought_lh_947b15' 'n_months_last_bought_ltc_43b9d5'
 'n_months_last_bought_grp_1581d7' 'n_months_last_bought_lh_507c37'
 'n_months_last_bought_lh_839f8a' 'n_months_last_bought_lh_e22a6a'
 'n_months_last_bought_grp_70e1dd' 'n_months_last_bought_lh_f852af'
 'n_months_last_bought_32c74c' 

In [ ]:
RFE_and_model_evaluation(RandomForestClassifier())

In [255]:
RFE_and_model_evaluation(KNeighborsClassifier())

ValueError: when `importance_getter=='auto'`, the underlying estimator KNeighborsClassifier should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.